In [1]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

aff058fe8244af0da38dbcf65bc4cdc0c602194d58ac95797f979579a2621a48


In [2]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 1067371 recs, Rejected: 0 recs in 0.355000 secs
1 rows returned.
Execution time: 460 ms, Total time: 460 ms
User admin disconnected from database omnisci


In [12]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT Country, SUM(Price) as price_sum FROM online_retail GROUP BY Country;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Country|price_sum
United Kingdom|4453169.287
France|67231.31000000001
USA|1755.88
Belgium|14766.88
Australia|8111.07
EIRE|125305.45
Germany|67564.45099999997
Portugal|17125.84
Japan|1258.61
Denmark|2397.890000000001
Nigeria|109.28
Netherlands|15663.23000000001
Poland|1940.22
Spain|20539.47
Channel Islands|9056.799999999999
Italy|8479.459999999999
Cyprus|6333.95
Greece|2600.73
Norway|41129.90999999999
Austria|4184.319999999999
Sweden|8709.920000000002
United Arab Emirates|2552.53
Finland|5234.610000000001
Switzerland|12049.88
Unspecified|3416.4
Malta|6575.589999999999
Bahrain|439.49
RSA|3353.85
Bermuda|84.70000000000002
Hong Kong|20956.87999999999
Singapore|25481.4
Thailand|227.97
Israel|1323.47
Lithuania|494.4100000000001
West Indies|122.77
Lebanon|358.61
Korea|150.24
Brazil|256.31
Canada|1058.09
Iceland|644.16
European Community|294.05
Saudi Arabia|24.11
Czech Republic|88.15000000000001
43 rows returned.
Execution time: 162 ms, Total time: 163 

In [11]:
%%bash
docker stop omnisci_test

omnisci_test
